In [ ]:
from model_create import ModelGenerator, clear_folders
clear_folders()
# Параметры, которые необходимо расчитать:
parameters = ['FOPT', 'WOPR:*', 'FPR', 'WBHP:P1',]# 'FWPT', 'FLPT', 'FGPT', 'FWIT']

# Зададим дату начала расчета и продолжительность:
start_date = u"1 'JAN' 2020"
mounths = 5 # количество итераций;
days = 5 # продолжительность итерации;

# Зададим размеры модели:
lgr = True # локальное измельчение сетки по центру;
cells_cy = 20 # количество измельченных ячеек по y;
cells_cx = 4 # количество измельченных ячеек по x;
cells_v = 10 # размер измельченных ячеек, м;
nx = 40 # количество ячеек по х;
ny = 40 # количество ячеек по y;
nz = 10 # количество ячеек по z;
lx = 1500 # длина модели по x, м;
ly = lx # длина модели по y, м;
dz = 1 # размер ячейки по z, м;
tops_depth = 2500 # глубина залегания пласта, м;

# Зададим пористость и проницаемость:
por = 0.3 # пористость, д.ед;

# Зададим название, расположение добываюещей скважины и ее режим работы:
horizontal = [True] # Идентификатор горизонтальной скважины для скрипта
prod_names = ['P1'] # название скважин;
prod_xs=[20] # координата скважин по x;
prod_ys=[11] # координата скважин по y (если horizontal=true - начальная координата);
y_stop = [31] # координата конца горизонтальной скважины, если horizontal=true;
prod_z1s=[1] # начало интервала вскрытия по z;
prod_z2s=[6] # конец интервала вскрытия по z (если horizontal=true, то показывает уровень вскрытия);
rezim = ['BHP'] # Режим работы скважины
prod_q_oil = ['*'] # дебит, м3/сут;
prod_bhp = ['200'] # забойное давление, атм;

# Другие настройки модели:
only_prod = True # Модель только с добывающими скважинами
upr_rezim_water = False # Моделируем упруго-водонапорный режим
upr_rezim_gas = False # Моделируем газонапорный режим
neogr_plast = False # Моделируем неограниченный пласт

# Задаем скин, радиус скважины (последовательно для добывающих и нагнетательных скважин):
skin = [0]
rw = [0.073]

# Свойства продукции:
oil_den = 860 # плотность нефти, кг/м3;
wat_den = 1010 # плотность воды, кг/м3;
gas_den = 0.9 # плотность газа, кг/м3;

# Свойства поороды:
rock_compr = 1.5E-005 # сжимаемость породы, Па^-1

# EQUILIBRIUM DATA:
p_depth = 2500 # Глубина замера пластового давления, м;
p_init = 320  # Начальное пластовое давление, атм;
o_w_contact = 2550 # Глубина ВНК, м;
pc_woc = 0 # Капиллярное давление на ВНК 
g_o_contact = 2400 # Глубина ГНК, м;
pc_goc = 0 # Капиллярное давление на ГНК

In [ ]:
#выбираем шаблон data файла для различных симуляторов
template = 1 # : 1-opm; 2-ecl (в разработке)
permx_list = [5, 50, 200] # проницаемость по x, мД;
kk = 0.1
model_list = []

for i in range(0, 3):
    permx = permx_list[i]
    permy = permx_list[i] # проницаемость по y, мД;
    permz = permx_list[i]*kk # проницаемость по z, мД;
    model_name = f'TEST_MODEL_HORIZONTAL.{i}'
    result_name = model_name
    model_list.append(' горизонтальная проницаемость - ' + str(permx) + ' мД')
    model = ModelGenerator(start_date=start_date, mounths=mounths,
                    days=days, nx=nx, ny=ny, nz=nz, dz=dz, por=por, permx=permx,
                    permy=permy, permz=permz, prod_names=prod_names, prod_xs=prod_xs,
                    prod_ys=prod_ys, prod_z1s=prod_z1s, prod_z2s=prod_z2s, prod_q_oil=prod_q_oil,
                    skin=skin, oil_den=oil_den, wat_den=wat_den, gas_den=gas_den, p_depth=p_depth, 
                    p_init=p_init, o_w_contact=o_w_contact, pc_woc=pc_woc, g_o_contact=g_o_contact,
                    pc_goc=pc_goc, tops_depth = tops_depth,
                    rezim=rezim, prod_bhp=prod_bhp, horizontal=horizontal, y_stop=y_stop, only_prod=only_prod,
                    lgr=lgr, lx=lx, ly=ly, cells_cy=cells_cy, cells_v=cells_v, cells_cx=cells_cx,
                    upr_rezim_water=upr_rezim_water, upr_rezim_gas=upr_rezim_gas, rw=rw, template=template, neogr=neogr_plast)
    model.create_model(model_name, result_name, parameters)

In [ ]:
import sys
import clr
import DLLs
from ctypes import *
import os

cur_path = os.path.abspath('') + 'dll'
sys.path.append(f'{cur_path}')
clr.AddReference('HMLW_net')

import HMLW_net
# написать гайд по установке pythonnet (пока только в windows)

In [ ]:
# Расчет продуктивности горизонатльной скважины на установившемся режиме
# Horizontal Well Productivity calculation (Ozkan model, 1989 and Joshi model, 1988)
# надо найти литературу
#model = 1 # 1-Ozkan; 2-Joshi;
def HWP(kk, h, re, rw, L, zw, model, mu, B, dP):
    if model == 1:
        print('Lmin: ' + str(round(8.6*kk**(-0.5)*h,1)))
        print('Lmax: ' + str(round(2*re,1)))
    elif model == 2:
        print('Lmin: ' + str(round(kk**(-0.5)*h,1)))
        print('Lmax: ' + str(round(1.8*re,1)))

    Jd = round(HMLW_net.HMLW_module.Jd_horizontal(rw, h, kk, L, zw, re, model),2)
    Sh = round(HMLW_net.HMLW_module.S_from_Jdss(Jd, rw, re),2)
    Qh = round(HMLW_net.HMLW_module.Q_from_Jd(Jd, kh, h, mu, B, dP),2)
    return Qh

In [ ]:
# метод для добавления данных на график
import pandas as pd 
import plotly.graph_objects as go
from plotly.offline import iplot
from ecl2df import pvt, EclFiles
import numpy as np
import scipy.interpolate

def figure_plot(filename, rw, kh, h, kk, L, zw, re, lx, ly, S, start_date, title):
    # Получаем забойное и пластовое давление на все иттерации
    start_date = pd.to_datetime(start_date)
    df_time = pd.read_csv(f'csv_folder/{filename}.csv', parse_dates=[0], index_col=[0], usecols=['Time' ,'FPR', 'WBHP:P1', 'WOPR:P1'])
    
    fig = go.Figure()
    trace_name = ['OPM', 'Батлер - протяженная гориз. скв. (147с.)']
    df_y = []
    df_y.append(df_time['WOPR:P1'])
    y_list = []
    for i in df_time.values:
        dP = i[1] - i[2]
        eclfiles = EclFiles(f'model_folder/{filename}')
        dframe = pvt.df(eclfiles)
        indic = dframe['KEYWORD'] == 'PVTO'
        res_df = dframe.loc[indic][['PRESSURE', 'VOLUMEFACTOR', 'VISCOSITY']]
        x = res_df['PRESSURE'].values
        y1 = res_df['VOLUMEFACTOR'].values
        z = np.poly1d(np.polyfit(x, y1, 2))
        xp = np.linspace(0, 600, 100)
        sol_list_x = [i[1], i[1]+0.1, i[1]+0.2, i[1]+0.3]
        sol_list_y = [0, 1, 2, 3]
        interp1 = scipy.interpolate.InterpolatedUnivariateSpline(xp, z(xp))
        interp2 = scipy.interpolate.InterpolatedUnivariateSpline(sol_list_x, sol_list_y)
        new_x = np.linspace(np.array(xp).min(), np.array(xp).max(), 100)
        new_y11 = interp1(new_x)
        new_y21 = interp2(new_x)
        idx1 = np.argwhere(np.diff(np.sign(new_y11 - new_y21)) != 0)
        y_sol = round(new_y11[idx1[0]][0].astype(float),2)
        x_sol = round(new_x[idx1[0]][0].astype(float),2)
        y2 = res_df['VISCOSITY'].values
        z = np.poly1d(np.polyfit(x, y2, 2))
        xp = np.linspace(0, 600, 100)
        sol_list_x = [i[1], i[1]+0.1, i[1]+0.2, i[1]+0.3]
        sol_list_y = [0, 1, 2, 3]
        interp1 = scipy.interpolate.InterpolatedUnivariateSpline(xp, z(xp))
        interp2 = scipy.interpolate.InterpolatedUnivariateSpline(sol_list_x, sol_list_y)
        new_x = np.linspace(np.array(xp).min(), np.array(xp).max(), 100)
        new_y12 = interp1(new_x)
        new_y22 = interp2(new_x)
        idx2 = np.argwhere(np.diff(np.sign(new_y12 - new_y22)) != 0)
        y_sol = round(new_y12[idx2[0]][0].astype(float),2)
        x_sol = round(new_x[idx2[0]][0].astype(float),2)
        y_list.append(bat_2(lx, dP, kh, new_y12[idx2[0]], kh*kk, rw, h, L, new_y11[idx1[0]])[0])
    df_y.append(y_list)
    for i in range(0, 2):
        fig.add_trace(go.Scatter(
                x=df_time.index,
                y=df_y[i],
                mode='lines',
                name=trace_name[i]))
    fig.update_xaxes(tickformat='%d.%m.%y')
    fig.update_layout(title=go.layout.Title(text=title),
                           xaxis_title='Дата',
                           yaxis_title='Qж, м3/сут')
    return fig

In [ ]:
filename = 'TEST_MODEL_HORIZONTAL.0'
rw_ = rw[0] # радиус скважины, м
kh = permx_list[0] # горизонтальная проницаемость, мд
h = nz*dz # мощность пласта, м
L = (y_stop[0]-prod_ys[0])*cells_v # длина горизонтального учатска, м
zw = h-prod_z2s[0] # расстояние от подошвы пласта до ствола, м
re = lx/2 # радиус контура питания, м
figure = figure_plot(filename, rw_, kh, h, kk, L, zw, re, lx, ly, skin[0], start_date, model_list[0])

iplot(figure)

In [ ]:
filename = 'TEST_MODEL_HORIZONTAL.1'
rw_ = rw[0] # радиус скважины, м
kh = permx_list[1] # горизонтальная проницаемость, мд
h = nz*dz # мощность пласта, м
L = (y_stop[0]-prod_ys[0])*cells_v # длина горизонтального учатска, м
zw = h-prod_z2s[0] # расстояние от подошвы пласта до ствола, м
re = lx/2 # радиус контура питания, м
figure = figure_plot(filename, rw_, kh, h, kk, L, zw, re, lx, ly, skin[0], start_date, model_list[1])

iplot(figure)

In [ ]:
filename = 'TEST_MODEL_HORIZONTAL.2'
rw_ = rw[0] # радиус скважины, м
kh = permx_list[2] # горизонтальная проницаемость, мд
h = nz*dz # мощность пласта, м
L = (y_stop[0]-prod_ys[0])*cells_v # длина горизонтального учатска, м
zw = h-prod_z2s[0] # расстояние от подошвы пласта до ствола, м
re = lx/2 # радиус контура питания, м
figure = figure_plot(filename, rw_, kh, h, kk, L, zw, re, lx, ly, skin[0], start_date, model_list[2])

iplot(figure)

In [ ]:
title = 'Динамика пластового давления по годам'
x_axis = 'Дата'
y_axis = 'Pпл, атм'
model.summ_plot(['FPR'], x_axis=x_axis, y_axis=y_axis, title=title, name=model_list)